### 전국일방통행표준데이터
- 데이터 전처리과정
시도명, 지정사유, 지정연도, 도로폭, 도로차로수, 보차분리여부

- 결측치 처리 (column 개별기준)
- 오류치 처리 (unit의 차이)
- 이상치 처리 (iqr 사용)
- 전처리 내용 정리

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
rd_data= []

rd_rowdata= pd.read_csv('전국일방통행도로표준데이터.csv', encoding ='cp949')
column_selection = ['시도명', '지정사유', '지정연도', '도로폭', '도로차로수', '보차분리여부']
rd_data = rd_rowdata[column_selection]


In [25]:
print(rd_rowdata.columns)
for e in rd_data: 
    print(e, '/t', rd_data[e].hasnans)

Index(['시도명', '시군구명', '도로명', '지정사유', '지정연도', '도로폭', '도로연장', '도로차로수', '보차분리여부',
       '시작점위도', '시작점경도', '종료점위도', '종료점경도', '도로안내표지일련번호', '관리기관명', '관리기관전화번호',
       '데이터기준일자', '제공기관코드', '제공기관명'],
      dtype='object')
시도명 /t False
지정사유 /t True
지정연도 /t True
도로폭 /t False
도로차로수 /t True
보차분리여부 /t False


In [26]:
#시도명

print(rd_data['시도명'].unique)
print(rd_data['시도명'].hasnans)
#결측치 없음
#오류치 : 10 ->'강원도'
#print(rd_data[rd_rowdata['시도명']=='10'])
filter1 = rd_data['시도명'] =='10'
#rd_data[filter1]['시도명'] = '강원도'   warning
rd_data.loc[filter1, '시도명'] = '강원도'

#이상치 : 없음
print(rd_data['시도명'].unique)
print(rd_data['시도명'].hasnans)

<bound method Series.unique of 0         경기도
1         경기도
2         경기도
3         경기도
4         경기도
        ...  
2154      경기도
2155      경기도
2156      경기도
2157      경기도
2158    서울특별시
Name: 시도명, Length: 2159, dtype: object>
False
<bound method Series.unique of 0         경기도
1         경기도
2         경기도
3         경기도
4         경기도
        ...  
2154      경기도
2155      경기도
2156      경기도
2157      경기도
2158    서울특별시
Name: 시도명, Length: 2159, dtype: object>
False


In [27]:
#지정사유
print(rd_data['지정사유'].unique)
print(rd_data['지정사유'].hasnans)

#결측치
na_filter = rd_data['지정사유'].isna()
print(na_filter.value_counts)
rd_data.loc[na_filter, '지정사유'] = '불분명'  #rd_data ['지정사유'] .filter('불분명')

#오류치 : 같은 내용 다름이름 정리
def e1(x):
    if '원활' in x:
        return '원활'
    elif '불편'in x:
        return '불편'
    elif '안전' in x:
        return '안전'
    elif '혼잡'  in x:
        return '혼잡'
    else:
        return x
rd_data.loc[:, '지정사유'] = rd_data.loc[:, '지정사유'].apply(e1)

#이상치: 없음
print(rd_data['지정사유'].unique)
print(rd_data['지정사유'].hasnans)

<bound method Series.unique of 0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
2154    통행원활
2155    통행원활
2156    통행원활
2157    통행원활
2158     NaN
Name: 지정사유, Length: 2159, dtype: object>
True
<bound method IndexOpsMixin.value_counts of 0        True
1        True
2        True
3        True
4        True
        ...  
2154    False
2155    False
2156    False
2157    False
2158     True
Name: 지정사유, Length: 2159, dtype: bool>
<bound method Series.unique of 0       불분명
1       불분명
2       불분명
3       불분명
4       불분명
       ... 
2154     원활
2155     원활
2156     원활
2157     원활
2158    불분명
Name: 지정사유, Length: 2159, dtype: object>
False


C:\Users\ITPS\AppData\Local\Temp\ipykernel_11632\2259414818.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rd_data.loc[:, '지정사유'] = rd_data.loc[:, '지정사유'].apply(e1)


In [28]:
#지정연도
print(rd_data['지정연도'].unique)
print(rd_data['지정연도'].hasnans)

#결측치 np.NaN -> 0
na_filter = rd_data['지정연도'].isna()
print(na_filter.value_counts())
rd_data.loc[na_filter, '지정연도'] = 0

#오류치, 이상치: 이상없음 



<bound method Series.unique of 0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2154   NaN
2155   NaN
2156   NaN
2157   NaN
2158   NaN
Name: 지정연도, Length: 2159, dtype: float64>
True
True     1609
False     550
Name: 지정연도, dtype: int64


In [29]:
#도로폭
print(rd_data['도로폭'].unique)
print(rd_data['도로폭'].hasnans)

#결측치, 오류치: 없음
#이상치: upper 보다 큰값을 np.NaN으로 교체, 후에 dropna() 이용 샘플 삭제
#print(rd_data['도로폭'].describe())
q1, q3= rd_data['도로폭'].quantile([0.25, 0.75])
iqr = q3 -q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr
print(lower, upper)
filter1 = rd_data['도로폭']> upper
print(filter1.value_counts())
rd_data.loc[filter1, '도로폭'] = np.NaN

<bound method Series.unique of 0       5.0
1       5.0
2       9.0
3       4.0
4       4.0
       ... 
2154    6.0
2155    6.0
2156    6.0
2157    6.0
2158    9.5
Name: 도로폭, Length: 2159, dtype: float64>
False
-0.5 11.5
False    2104
True       55
Name: 도로폭, dtype: int64


In [30]:
#도로차로수
print(rd_data['도로차로수'].unique)
print(rd_data['도로차로수'].hasnans)

#결측치:  1개 샘플, 후에 dropna()을 이용 제거
na_filter = rd_data['도로차로수'].isna()
print(na_filter.value_counts())

#오류치: 60  -> np.NaN 대체, 후에 dropna()이용 제거
filter1 = rd_data['도로차로수'] == 60
print(filter1.value_counts())
rd_data.loc[filter1, '도로차로수'] = np.NaN

#이상치
print(rd_data['도로차로수'].unique)
print(rd_data['도로차로수'].hasnans)






<bound method Series.unique of 0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2154    1.0
2155    1.0
2156    1.0
2157    1.0
2158    1.0
Name: 도로차로수, Length: 2159, dtype: float64>
True
False    2158
True        1
Name: 도로차로수, dtype: int64
False    2158
True        1
Name: 도로차로수, dtype: int64
<bound method Series.unique of 0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2154    1.0
2155    1.0
2156    1.0
2157    1.0
2158    1.0
Name: 도로차로수, Length: 2159, dtype: float64>
True


In [31]:
#보차분리여부
print(rd_data['보차분리여부'].unique)
print(rd_data['보차분리여부'].hasnans)

#결측치: 1개 ' 결측치 -> np.NaN으로 대체, 후에 dropna()이용 삭제
na_filter = rd_data['보차분리여부'] ==' '
print(na_filter.value_counts())
rd_data.loc[na_filter, '보차분리여부'] = np.NaN

#오류치, 이상치 없음
print(rd_data['보차분리여부'].unique)
print(rd_data['보차분리여부'].hasnans)



<bound method Series.unique of 0       Y
1       Y
2       Y
3       Y
4       Y
       ..
2154    N
2155    N
2156    N
2157    N
2158    N
Name: 보차분리여부, Length: 2159, dtype: object>
False
False    2158
True        1
Name: 보차분리여부, dtype: int64
<bound method Series.unique of 0       Y
1       Y
2       Y
3       Y
4       Y
       ..
2154    N
2155    N
2156    N
2157    N
2158    N
Name: 보차분리여부, Length: 2159, dtype: object>
True


In [32]:
rd_data_f = rd_data.dropna()

print(rd_data.shape)
print(rd_data_f.shape)

(2103, 6)
(2159, 6)
